# Bicycle Mobility Flow Analysis

## TemBici - São Paulo

Our approach is based on dividing the city into homogeneous regions by using a uniform **grid** and counting the number of bike trips from one grid cell to the other (called here a **flow**). We draw directed arrows to show flow direction and adjust the origin and end point of flows according to a weighted average based on the dock station usage for that specific flow.

The raw amount of flows within a city is very large. Showing all of them to a user is overwhelming and does not allow any reasonable analysis. To show this information in a comprehensible manner, we divide the flows in **tiers**. For instance, dividing the flows into 4 tiers, each one will contain 25\% of the trips.

In addition to the mobility flows, another relevant information is what regions of the city are the major hubs initiating or ending bike trips. The **hub** analysis map shows dark green markers in the top regions of the city where bike trips start and dark red markers in the top regions where trips end. Light green markers and light red markers point to second tier hubs.

In [1]:
!pip install geopandas

In [2]:
import tembici.load_trips as tr
import tembici.stations as st

import bikescience.sp_grid as gr
from bikescience.stations import draw_stations
import bikescience.interface as interf
import bikescience.tiers as tiers
import bikescience.load_trips as btr
import bikescience.flow as flow
from bikescience.arrow import draw_arrow

import geopandas as gpd
import json
import pandas as pd
from ipywidgets import interact_manual, widgets
from IPython.core.display import display, HTML
import folium
from folium.plugins import HeatMap
import warnings
import requests
warnings.simplefilter('ignore')

/tmp/ipykernel_4615/395690581.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Input data:
* all trips files
* bike stations
* distances between stations (calculated by [GraphHopper](https://www.graphhopper.com/) service)

In [3]:
# Input data
year = '2021'
data_folder = '../data/'
file_filter = data_folder + 'trips/loaded_trips/trips_'+year+'.csv'
trips = pd.read_csv(file_filter)
trips.head()

,date,Index,tripduration,start_station_name_old,index_start,starttime,end_station_name_old,index_end,stoptime,per_day,...,holiday,trip_id,birth_year,initial_station_latitude,initial_station_longitude,final_station_latitude,final_station_longitude,age,start_station_name,end_station_name
0,2020-01-01,4758319,3596,76 - 8° Batalhão,76.0,2020-01-01 16:14:43,91 - Paulista 778,91.0,2020-01-01 17:14:39,2020-01-01,...,True,5885062BikeSampa,1996-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,24.0,8º Batalhão,Paulista 778
1,2020-01-01,4758337,2922,21 - Parque do Povo portão 1,21.0,2020-01-01 16:18:08,21 - Parque do Povo portão 1,21.0,2020-01-01 17:06:50,2020-01-01,...,True,5885083BikeSampa,NaN,NaN,NaN,NaN,NaN,NaN,Parque do Povo portão 1,Parque do Povo portão 1
2,2020-01-01,4758338,199,304 - Gandavo,304.0,2020-01-01 16:18:10,304 - Gandavo,304.0,2020-01-01 16:21:29,2020-01-01,...,True,5885111BikeSampa,1997-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,23.0,Gandavo,Gandavo
3,2020-01-01,4758339,4151,131 - Pq. Ibirapuera Portão 06,131.0,2020-01-01 16:18:19,43 - Auditório Ibirapuera,43.0,2020-01-01 17:27:30,2020-01-01,...,True,5885086BikeSampa,1984-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,36.0,Ibirapuera Portão 06,Herois da FEB
4,2020-01-01,4758340,6065,198 - Marechal Deodoro,198.0,2020-01-01 16:18:25,196 - Alameda Barros,196.0,2020-01-01 17:59:30,2020-01-01,...,True,5885091BikeSampa,1998-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,22.0,Marechal Deodoro,Alameda Barros


In [4]:
trips['starttime'] = pd.to_datetime(trips['starttime'])
trips['stoptime'] = pd.to_datetime(trips['stoptime'])

In [5]:
stations = pd.read_csv(data_folder + 'stations/treated_data/2021.11.08_Endereços-BikeSAMPA.csv')
# stations = stations.drop(['Unnamed: 0'], axis = 1)
stations

,id,name,address,lat,lon,VAGAS/DOCK AGORA,DATA INAUGURAÇÃO
0,1,Largo da Batata,"Av. Brigadeiro Faria Lima, esquina R. Teodoro ...",-23.566831,-46.693741,83,2018-01-30
1,2,SESC Pinheiros,"R. Ferreira de Araújo, em frente ao número 1031",-23.566920,-46.698190,15,2020-12-19
2,3,CPTM Pinheiros,"R. Gilberto Sabino, 138/ ao lado do ponto de ô...",-23.566478,-46.701258,15,2018-01-30
3,4,Rua Diogo Moreira,"Av. Brigadeiro Faria Lima, na altura do número...",-23.569145,-46.692003,23,2018-05-10
4,5,Chicão Vive,"Rua Butantã , 192",-23.569894,-46.697897,7,2019-12-03
...,...,...,...,...,...,...,...
254,347,Metrô Brooklin,"Av. Santo Amaro, próximo ao acesso do metrô Br...",-23.626894,-46.687573,29,2018-12-12
255,354,Praça Oswaldo Cruz,"Praça Oswaldo Cruz, oposto 97",-23.571604,-46.644266,11,2019-12-21
256,355,R. Dr. Rafael de Barros,"Rua Dr. Rafael de Barros, em frente ao 253 / P...",-23.572712,-46.646530,19,2019-01-31
257,358,Shopping Villa Lobos,"Avenida das Nações Unidas, 4777",-23.551665,-46.722563,7,2018-12-12


In [6]:
stations = pd.read_csv(data_folder + 'stations/treated_data/2021.11.08_Endereços-BikeSAMPA.csv')
stations = st.stations_geodf(stations)

stations_distances = pd.read_csv(data_folder + 'stations/distance/stations_distance.csv')

In [7]:
# SIRGAS 2000 / UTM zone 23S
# http://www.processamentodigital.com.br/2013/07/27/lista-dos-codigos-epsg-mais-utilizados-no-brasil/

# CET: http://cetsp1.cetsp.com.br/mapabasico/export/shapesInfraestruturaCicloviaria.zip

bike_lanes_ciclorrotas = \
        gpd.read_file(data_folder + 'infrastructure/geosampa/SIRGAS_SHP_redecicloviaria_new/Ciclorrotas.shp')
bike_lanes_ciclorrotas.crs = {'init': 'epsg:31983'}  
bike_lanes_ciclorrotas.to_crs(epsg='4326', inplace=True)

bike_lanes_ciclovias = \
        gpd.read_file(data_folder + 'infrastructure/geosampa/SIRGAS_SHP_redecicloviaria_new/Ciclovias.shp')
bike_lanes_ciclovias.crs = {'init': 'epsg:31983'}  
bike_lanes_ciclovias.to_crs(epsg='4326', inplace=True)

subway_stops = \
        gpd.read_file(data_folder + 'infrastructure/geosampa/SIRGAS_SHP_estacaometro/SIRGAS_SHP_estacaometro_point.shp')
subway_stops.crs = {'init': 'epsg:31983'}
subway_stops.to_crs({'init': 'epsg:4326'}, inplace=True)

rail_ferry_stops = \
        gpd.read_file(data_folder + 'infrastructure/geosampa/SIRGAS_SHP_estacaotrem/SIRGAS_SHP_estacaotrem_point.shp')
rail_ferry_stops.crs = {'init': 'epsg:31983'}
rail_ferry_stops.to_crs({'init': 'epsg:4326'}, inplace=True)

bus_stops = \
        gpd.read_file(data_folder + 'infrastructure/geosampa/SIRGAS_SHP_pontoonibus/SIRGAS_SHP_pontoonibus.shp')
bus_stops.crs = {'init': 'epsg:31983'}
bus_stops.to_crs({'init': 'epsg:4326'}, inplace=True)

In [8]:
bike_lanes_ciclorrotas

,programa,inauguracao,extensao_t,extensao_c,geometry
0,CICLORROTA BATURITÉ/ DIAMANTE,2021-05-18,83,548,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
1,CICLORROTA BATURITÉ/ DIAMANTE,2021-05-18,465,548,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
2,CICLORROTA BROOKLIN,2011-07-20,911,5950,"LINESTRING (-135.23564 -85.52570, -135.23565 -..."
3,CICLORROTA BROOKLIN,2011-07-20,279,5950,"LINESTRING (-135.23565 -85.52570, -135.23565 -..."
4,CICLORROTA BROOKLIN,2011-07-20,227,5950,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
...,...,...,...,...,...
112,CICLORROTA VILA MARIANA,2012-05-24,540,4953,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
113,CICLORROTA VILA MARIANA,2012-05-24,170,4953,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
114,CICLORROTA VILA MARIANA,2012-05-24,121,4953,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
115,CICLORROTA VILA MARIANA,2012-05-24,103,4953,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."


In [9]:
bike_lanes_ciclovias

,programa,inauguracao,extensao_t,extensao_c,geometry
0,CICLOFAIXA CAMINHO VERDE,2008-09-27,1926,11582,"LINESTRING (-135.23563 -85.52570, -135.23563 -..."
1,CICLOFAIXA CAMINHO VERDE,2008-09-27,2394,11582,"LINESTRING (-135.23563 -85.52570, -135.23563 -..."
2,CICLOFAIXA CAMINHO VERDE,2008-09-27,3689,11582,"LINESTRING (-135.23563 -85.52570, -135.23563 -..."
3,CICLOFAIXA CAMINHO VERDE,2008-09-27,1607,11582,"LINESTRING (-135.23563 -85.52570, -135.23563 -..."
4,CICLOFAIXA CAMINHO VERDE,2008-09-27,1966,11582,"LINESTRING (-135.23563 -85.52570, -135.23563 -..."
...,...,...,...,...,...
1817,CICLOFAIXA PACAEMBU / FAAP / MACKENZIE,2014-09-06,244,1553,"LINESTRING (-135.23563 -85.52570, -135.23564 -..."
1818,CICLOFAIXA PACAEMBU / FAAP / MACKENZIE,2014-09-06,184,1553,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
1819,CICLOFAIXA PACAEMBU / FAAP / MACKENZIE,2014-09-06,97,1553,"LINESTRING (-135.23564 -85.52570, -135.23564 -..."
1820,CICLOFAIXA PACAEMBU / FAAP / MACKENZIE,2014-09-06,52,1553,"LINESTRING (-135.23563 -85.52570, -135.23563 -..."


In [10]:
# Infrastructure plotting

protected_color = 'red'   # ciclovias
sharrow_color = 'orange'  # ciclorrotas
trail_color = 'green'     # ciclofaixas
bike_station_color = 'black'
subway_color = 'brown'
rail_color = 'lime'
bus_color = 'gray'

style_grid = lambda x: {'color': 'black', 'weight': 1, 'opacity': 0.3, 'fillOpacity': 0.0}
style_sharrow = lambda style:{'color':sharrow_color, 'weight': 2}
style_protected = lambda style:{'color':protected_color, 'weight': 2}
style_trail = lambda style:{'color':trail_color, 'weight': 2}

def plot_grid(fmap, grid):
    folium.GeoJson(grid.geodataframe().to_json(), name='Grid', style_function=style_grid).add_to(fmap)
    
def plot_cycling_infra(fmap):
    # CHANGEEE
    folium.GeoJson(bike_lanes_ciclorrotas,
                   style_function=style_sharrow,
                   name='Ciclorrota').add_to(fmap)
    
    folium.GeoJson(bike_lanes_ciclovias,
                   style_function=style_sharrow,
                   name='Ciclovia').add_to(fmap)
    
    # folium.GeoJson(bike_lanes.loc[bike_lanes['rc_tipo']=='ciclorrota'],
    #                style_function=style_sharrow,
    #                name='').add_to(fmap)
    # folium.GeoJson(bike_lanes.loc[bike_lanes['rc_tipo']=='ciclovia'],
    #                style_function=style_protected,
    #                name='Ciclovia').add_to(fmap)
    # folium.GeoJson(bike_lanes.loc[bike_lanes['rc_tipo']=='ciclofaixa'],
    #                style_function=style_trail,
    #                name='Ciclofaixa').add_to(fmap)

def plot_bike_stations(fmap):
    bike_stations = folium.FeatureGroup(name='Bike stations')
    for index, row in stations.iterrows():
        bike_stations.add_child(folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=3,
                                popup=row['name'], color=bike_station_color))
    fmap.add_child(bike_stations)
    
def plot_subway_rail_stops(fmap):
    subway_stops_g = folium.FeatureGroup(name='Subway stops')
    for index, row in subway_stops.iterrows():
        subway_stops_g.add_child(folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=3,
                                popup="", color=subway_color))
    fmap.add_child(subway_stops_g)

    rail_stops_g = folium.FeatureGroup(name='Train stops')
    for index, row in rail_ferry_stops.iterrows():
        rail_stops_g.add_child(folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=3,
                                popup="", color=rail_color))
    fmap.add_child(rail_stops_g)
    
def plot_bus_stops(fmap):
    bus_stops_g = folium.FeatureGroup(name='Bus stops',show=False)
    for index, row in bus_stops.iterrows():
        bus_stops_g.add_child(folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=1,
                                popup="", color=bus_color))
    fmap.add_child(bus_stops_g)

## Grids, Flows and Tiers

### Grids

Grid cells at different granularity levels representing small areas of a city. The cells usually contain a couple of bike stations.

In [11]:
# default
grid = gr.create(n=50, 
                 west_offset=-0.11, east_offset=0.05, north_offset=0.05, south_offset=-0.1)

def set_grid_limits(west_delta, east_delta, north_delta, south_delta, grid_size):
    global grid
    grid = gr.create(n=grid_size, 
                     west_offset=west_delta, east_offset=east_delta, 
                     north_offset=north_delta, south_offset=south_delta)
    fmap = grid.map_around(zoom=12)
    draw_stations(fmap, stations, 'name')
    folium.Marker([gr.SP_LAT, gr.SP_LON]).add_to(fmap)
    display(fmap)
    
im = interact_manual(
    set_grid_limits,
    west_delta=interf.grid_delta_selector(-0.11, -0.5, 0.5),
    east_delta=interf.grid_delta_selector(0.05, -0.5, 0.5),
    north_delta=interf.grid_delta_selector(0.05, -0.5, 0.5),
    south_delta=interf.grid_delta_selector(-0.1, -0.5, 0.5),
    grid_size=widgets.SelectionSlider(options=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100], value=50)
)
im.widget.children[5].description = 'Show grid'

interactive(children=(FloatSlider(value=-0.11, description='west_delta', layout=Layout(width='50%'), max=0.5, …

### Flows

Trip flows between two grid cells. Each flow is a set of trips from the origin to the destination cell.

### Tiers  

Distribution of flows of trips across _N_ tiers. Flows are:
* ordered by number of trips, descending
* broken into _N_ quantiles
* each quantile has some information summarized

In [12]:
trips

,date,Index,tripduration,start_station_name_old,index_start,starttime,end_station_name_old,index_end,stoptime,per_day,...,holiday,trip_id,birth_year,initial_station_latitude,initial_station_longitude,final_station_latitude,final_station_longitude,age,start_station_name,end_station_name
0,2020-01-01,4758319,3596,76 - 8° Batalhão,76.0,2020-01-01 16:14:43,91 - Paulista 778,91.0,2020-01-01 17:14:39,2020-01-01,...,True,5885062BikeSampa,1996-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,24.0,8º Batalhão,Paulista 778
1,2020-01-01,4758337,2922,21 - Parque do Povo portão 1,21.0,2020-01-01 16:18:08,21 - Parque do Povo portão 1,21.0,2020-01-01 17:06:50,2020-01-01,...,True,5885083BikeSampa,NaN,NaN,NaN,NaN,NaN,NaN,Parque do Povo portão 1,Parque do Povo portão 1
2,2020-01-01,4758338,199,304 - Gandavo,304.0,2020-01-01 16:18:10,304 - Gandavo,304.0,2020-01-01 16:21:29,2020-01-01,...,True,5885111BikeSampa,1997-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,23.0,Gandavo,Gandavo
3,2020-01-01,4758339,4151,131 - Pq. Ibirapuera Portão 06,131.0,2020-01-01 16:18:19,43 - Auditório Ibirapuera,43.0,2020-01-01 17:27:30,2020-01-01,...,True,5885086BikeSampa,1984-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,36.0,Ibirapuera Portão 06,Herois da FEB
4,2020-01-01,4758340,6065,198 - Marechal Deodoro,198.0,2020-01-01 16:18:25,196 - Alameda Barros,196.0,2020-01-01 17:59:30,2020-01-01,...,True,5885091BikeSampa,1998-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,22.0,Marechal Deodoro,Alameda Barros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354962,2020-12-31,4478491,2817,96 - Tutóia,96.0,2020-12-31 16:19:00,74 - Círculo Militar,74.0,2020-12-31 17:05:57,2020-12-31,...,False,9077382BikeSampa,1987-01-01,-23.573823,-46.651414,-23.580154,-46.656249,33.0,Tutóia,Círculo Militar
2354963,2020-12-31,4478371,5870,146 - Ibirapuera portão 03,146.0,2020-12-31 10:58:34,146 - Ibirapuera portão 03,146.0,2020-12-31 12:36:24,2020-12-31,...,False,9075846BikeSampa,1987-01-01,-23.589043,-46.652787,-23.589043,-46.652787,33.0,Ibirapuera Portão 03,Ibirapuera Portão 03
2354964,2020-12-31,4479320,10486,347 - Metrô Brooklin,347.0,2020-12-31 11:41:36,25 - Rua Helena,25.0,2020-12-31 14:36:22,2020-12-31,...,False,9076109BikeSampa,1978-01-01,-23.626894,-46.687573,-23.593063,-46.685352,42.0,Metrô Brooklin,Rua Helena
2354965,2020-12-31,4478800,349,224 - Metrô Fradique Coutinho,224.0,2020-12-31 11:40:46,224 - Metrô Fradique Coutinho,224.0,2020-12-31 11:46:35,2020-12-31,...,False,9076102BikeSampa,1998-01-01,-23.565532,-46.684883,-23.565532,-46.684883,22.0,Metrô Fradique Coutinho,Metrô Fradique Coutinho


In [16]:
the_grid = None
od = None
trips_filter = None

def show_map(period, days, period_of_day, distance, grid_size, tier):
    global the_grid, od, trips_filter
    # print('trips_filter 1')
    # print(trips_filter)
    the_grid = gr.create(n=grid_size, 
                         west_offset=-0.11, east_offset=0.05, north_offset=0.05, south_offset=-0.1)
    fmap = the_grid.map_around(zoom=13)
    print('Calculating...')
    
    plot_cycling_infra(fmap)
    
    # flows
    start, end = interf.period_interval(period)
    trips_filter = btr.day_functions[days](trips)
    # print('trips_filter 2')
    # print(trips_filter)

    trips_filter = btr.period_functions[period_of_day](trips_filter)

    # print('trips_filter 3')
    # print(trips_filter)

    trips_filter = trips_filter[(trips_filter['starttime'] >= start) & (trips_filter['starttime'] < end)]

    # print('trips_filter 4')
    # print(trips_filter)

    if distance in [1, 2]:
        trips_filter = st.merge_trips_and_stations(trips_filter, stations)
        trips_filter = st.merge_trips_stations_and_distances(trips_filter, stations_distances)

        # print('trips_filter 5')
        # print(trips_filter)

        if distance == 1:
            trips_filter = trips_filter[trips_filter['distance'] < 1]
        else:
            trips_filter = trips_filter[trips_filter['distance'] > 4]

    if len(trips_filter) == 0:
        print('No trips found.')
        return
            
    od = flow.od_countings(trips_filter, the_grid, stations,
                           station_index='name', 
                           start_station_index='start_station_name', 
                           end_station_index='end_station_name')
    
    # print('od')
    # print(od)
    
    flow.draw_stations(fmap, stations, 'name')
    
    tiers_table, _ = tiers.separate_into_tiers(od.sort_values('trip counts', ascending=False), trips_filter, None, 
                                               max_tiers=4)
    display(tiers_table)

    tiers_table.to_csv('tiers/tiers_table_'+year+'.csv')

    if tier > 0:
        tiers_row = tiers_table[tiers_table['tier'] == tier]
        tiers_row = tiers_row.loc[tiers_row.index[0]]
        flow.flow_map(fmap, od, the_grid, stations, minimum=tiers_row['min'], maximum=tiers_row['top'], radius=2.0,
                     text=flow.POPUP_FLOW_ID)
    else:
        tiers_row = tiers_table[tiers_table['tier'] == 2]
        tiers_row = tiers_row.loc[tiers_row.index[0]]
        flow.flow_map(fmap, od, the_grid, stations, minimum=tiers_row['min'], radius=2.0, text=flow.POPUP_FLOW_ID)
        
    print('Done.')
    file = 'maps/flows'+year+'.html'
    fmap.save(file)
    display(HTML('Saved at <a href="' + file + '" target="_blank">' + file + '</a>'))
    display(fmap)

flow.N = 20
min_index = 0
max_index = 11
if year == '2022':
    max_index = 3
im = interact_manual(
    show_map,
    period=interf.period_selector(trips, index=(min_index, max_index)),
    days=widgets.Dropdown(options=[('all', 0), ('working days', 1), ('weekends', 2), ('holidays', 3), 
                                   ('weekends + holidays', 4)], value=1),
    period_of_day=widgets.Dropdown(options=[('all', 0), ('morning', 1), ('lunchtime', 2), ('afternoon', 3)],
                                   value=1),
    distance=widgets.Dropdown(options=[('all', 0), ('< 1Km', 1), ('> 4Km', 2)], value=0),
    grid_size=widgets.SelectionSlider(options=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100], value=50),
    tier=widgets.Dropdown(options=[('4', 4), ('3', 3), ('2', 2), ('all', 0)], value=0)
)
im.widget.children[6].description = 'Show map'

interactive(children=(SelectionRangeSlider(continuous_update=False, description='Trip period', index=(0, 11), …

## Select a flow to calculate the route:

#### Some examples are:
* 27 17 18 20 - faria lima - morning, 01/18-06/19, tier 4, all distances, working days, grid 50
* 26 29 18 18 - jd. paulista vl olimpia - morning, 02/19-06/19, tier 3,  >4km, working days, grid 50
* 26 32 18 27 - paraiso ibirapuera - morning, 01/19-06/19, tier 4, all distances, weekend, grid 50
* 9 28 29 5 - plto paulista ime usp

#### Colors:
* gray: on foot
* black: on bike

In [14]:
def show_route(start_i, start_j, end_i, end_j, grid, cycling_infrastructure, bike_stations, 
               subway_train_stops, bus_stops):
    global the_grid, od, trips_filter

    print('od')
    print(od)

    if the_grid == None:
        display(HTML('Please run the cell above and select a flow to show its identifiers.'))
        return
    print('Calculating...')
    fmap = the_grid.map_around(zoom=13)
    
    # plot accessories
    if grid: plot_grid(fmap, the_grid)
    if cycling_infrastructure: plot_cycling_infra(fmap)
    if bike_stations: plot_bike_stations(fmap)
    if subway_train_stops: plot_subway_rail_stops(fmap)
    if bus_stops: plot_bus_stops(fmap)
    
    # plot the flow
    the_flow = od[(od['i_start'] == start_i) & (od['j_start'] == start_j) &
                  (od['i_end'] == end_i) & (od['j_end'] == end_j)]
    
    print('the_flow')
    print(the_flow)
    
    if len(the_flow) == 0:
        print('Invalid route, click on a flow in the map above.')
        return
    
    the_flow = the_flow.loc[the_flow.index[0]]
    draw_arrow(fmap, the_flow['origin'].y, the_flow['origin'].x, the_flow['destination'].y, the_flow['destination'].x, radius_fac=2.0)
    
    # query GraphHopper for a bike route (Google requires credit card activation)
    gh_key = 'ddafcee1-4219-427f-81e0-f564d4ff2e8c'
    """
    gh_url = 'https://graphhopper.com/api/1/route?key=' + gh_key + \
             '&point=' + str(the_flow['origin'].y) + ',' + str(the_flow['origin'].x) + \
             '&point=' + str(the_flow['destination'].y) + ',' + str(the_flow['destination'].x) + \
             '&vehicle=bike&debug=false&type=json&points_encoded=false'
    """
    gh_url = 'https://graphhopper.com/api/1/route?key=' + gh_key + \
             '&point={},{}' + \
             '&point={},{}' + \
             '&vehicle={}&debug=false&type=json&points_encoded=false'
    
    req = requests.get(gh_url.format(the_flow['origin'].y, the_flow['origin'].x, 
                                     the_flow['destination'].y, the_flow['destination'].x, 'foot'))
    print('GraphHopper service (foot):', req.status_code, req.reason)
    paths = req.json()['paths']
    for p in paths: 
        folium.GeoJson(p['points'], 
                       style_function=lambda x: {'dashArray': '5 10', 'color': 'gray', 'weight': 6}).add_to(fmap)

    req = requests.get(gh_url.format(the_flow['origin'].y, the_flow['origin'].x, 
                                     the_flow['destination'].y, the_flow['destination'].x, 'bike'))
    print('GraphHopper service (bike):', req.status_code, req.reason)
    paths = req.json()['paths']
    for p in paths: 
        folium.GeoJson(p['points'], 
                       style_function=lambda x: {'dashArray': '5 10', 'color': 'black', 'weight': 5}).add_to(fmap)

    print('Done.')
    route_file = 'maps/route.html'
    fmap.save(route_file)
    display(fmap)
    
im = interact_manual(
    show_route,
    start_i=widgets.IntText(),
    start_j=widgets.IntText(),
    end_i=widgets.IntText(),
    end_j=widgets.IntText(),
    grid=True,
    cycling_infrastructure=True,
    bike_stations=True, 
    subway_train_stops=True,
    bus_stops=False
)
im.widget.children[9].description = 'Show route'

interactive(children=(IntText(value=0, description='start_i'), IntText(value=0, description='start_j'), IntTex…

### Hub analysis

Concentration of trip starts and ends.

In [15]:
def show_heatmap(period, trip_point, days, period_of_day):
    print('Calculating...')
    global the_grid, od, trips_filter
    heatmap = folium.Map([(the_grid.north_limit+the_grid.south_limit)/2, (the_grid.west_limit+the_grid.east_limit)/2], 
                         zoom_start=12, tiles='stamentoner')
    
    start, end = interf.period_interval(period)
    trips_filter = btr.day_functions[days](trips)
    trips_filter = btr.period_functions[period_of_day](trips_filter)
    trips_filter = trips_filter[(trips_filter['starttime'] >= start) & (trips_filter['starttime'] < end)]
    trips_filter = st.merge_trips_and_stations(trips_filter, stations)
    
    heat_data = [[row['lat_' + trip_point], row['lon_' + trip_point]] 
                 for index, row in trips_filter.iterrows()]
    HeatMap(heat_data, blur=25, max_val=40, min_opacity=0.6).add_to(heatmap)
    heatmap.save("maps/heatmap.html")
    print('Done.')
    display(heatmap)

im = interact_manual(
    show_heatmap,
    period=interf.period_selector(trips, index=(0, 5)), 
    trip_point=widgets.RadioButtons(options=['start', 'end']),
    days=widgets.Dropdown(options=[('all', 0), ('working days', 1), ('weekend', 2)], value=1),
    period_of_day=widgets.Dropdown(options=[('all', 0), ('morning', 1), ('lunchtime', 2), ('afternoon', 3)],
                                   value=1)
)
im.widget.children[4].description = 'Show map'

interactive(children=(SelectionRangeSlider(continuous_update=False, description='Trip period', index=(0, 5), l…